关于不同存储精度问题可以查看[《LLM大模型之精度问题（FP16，FP32，BF16）详解与实践》](https://zhuanlan.zhihu.com/p/657886517) 。这里我们主要研究两个问题：
1. 模型不同精度下显存占用情况；
2. 模型不同精度之间如何转换；

以显卡NVIDIA 4050 6G，模型用Qwen2-0.5B为例，这个模型的保存的精度通过查看模型文件的congfig.json可以看到是"torch_dtype": "float16"。

In [1]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 打印版本号
print("transformers version:", transformers.__version__)
print("torch version:", torch.__version__)

# 检查系统中是否有可用的 GPU
if torch.cuda.is_available():
    # 获取可用的 GPU 设备数量
    num_devices = torch.cuda.device_count()
    print("可用 GPU 数量:", num_devices)

    # 遍历所有可用的 GPU 设备并打印详细信息
    for i in range(num_devices):
        device = torch.cuda.get_device_properties(i)
        print(f"\nGPU {i} 的详细信息:")
        print("名称:", device.name)
        print("计算能力:", f"{device.major}.{device.minor}")
        print("内存总量 (GB):", round(device.total_memory / (1024**3), 1))
else:
    print("没有可用的 GPU")

transformers version: 4.41.0
torch version: 2.1.1+cu118
可用 GPU 数量: 1

GPU 0 的详细信息:
名称: NVIDIA GeForce RTX 4050 Laptop GPU
计算能力: 8.9
内存总量 (GB): 6.0


In [2]:
# 加载模型
model_name = "Qwen2-0.5B-Instruct" # 你模型存放的位置
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cuda:0", torch_dtype=torch.float16)
# 打印参数
total_parameters = model.num_parameters()
print("Total parameters in the model:", total_parameters)

Total parameters in the model: 494032768


以float16进行加载，也就是每个参数16个bit，2个byte，计算一下这么多参数应该占多少显存：

In [3]:
# 计算每个参数的大小（以字节为单位）
size_per_parameter_bytes = 2
# 计算模型在显存中的总空间（以字节为单位）
total_memory_bytes = total_parameters * size_per_parameter_bytes
# 将字节转换为更常见的单位（GB）
total_memory_gb = total_memory_bytes / (1024**3)

print("Total memory occupied by the model in MB:", total_memory_gb)

Total memory occupied by the model in MB: 0.9202077388763428


接着看一下，现在的显存占用：

In [4]:
# 计算模型的显存占用
memory_allocated = torch.cuda.memory_allocated(device='cuda:0')
# 将字节转换为更常见的单位（GB）
memory_allocated_gb = memory_allocated / (1024**3)

print("Memory allocated by the model in GB:", memory_allocated_gb)

Memory allocated by the model in GB: 1.115407943725586


可以看到这数值是接近一致的（稍微差一点的原因：框架torch以及transformers本身、GPU的本身缓存等等）。以上是在float16下加载的，同理看一下在float32下加载的情况：

In [6]:
# 加载模型float32
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cuda:0", torch_dtype=torch.float32)

# 打印参数
total_parameters = model.num_parameters()
print("Total parameters in the model:", total_parameters)

# 计算每个参数的大小（以字节为单位）
size_per_parameter_bytes = 2
# 计算模型在显存中的总空间（以字节为单位）
total_memory_bytes = total_parameters * size_per_parameter_bytes
# 将字节转换为更常见的单位（GB）
total_memory_gb = total_memory_bytes / (1024**3)
print("Total memory occupied by the model in MB:", total_memory_gb)

# 计算模型的显存占用
memory_allocated = torch.cuda.memory_allocated(device='cuda:0')
# 将字节转换为更常见的单位（GB）
memory_allocated_gb = memory_allocated / (1024**3)
print("Memory allocated by the model in GB:", memory_allocated_gb)

Total parameters in the model: 494032768
Total memory occupied by the model in MB: 0.9202077388763428
Memory allocated by the model in GB: 2.2383761405944824


参考：https://zhuanlan.zhihu.com/p/658343628